## **What does this code do?**
- Pulls data from Fingertips using a formatted and parsed URL. The parameters by which the data is pulled are fully customisable
- Performs basic data transformation steps on the Fingertips data, such as filtering for NCL and removing undesired columns
- Retrieves social prescribing data from raw data folder in Sharepoint (pre-downloaded and uploaded manually from email)
- Performs data transformation steps on the social prescribing data, removing rows, creating a date column etc.
- Merges the Fingertips and social prescribing data
- Uploads the merged and finalised file to the production data folder in Sharepoint

In [1]:
import fingertips_py as ftp
import pandas as pd

In [2]:
# df = ftp.api_calls.deal_with_url_error("https://fingertipsws.phe.org.uk/api/all_data/csv/by_group_id?v=/0-2205ecc6/&parent_area_code=nE38000240&parent_area_type_id=66&child_area_type_id=7&group_id=1938133365&category_area_code=null")

## Retrieving Fingertips Data

In [3]:
ftp.metadata.get_metadata(indicator_ids=None, domain_ids=None, profile_ids=92) #pulls metadata on cancer profile

,Indicator ID,Indicator,Definition,Rationale,Data source,Indicator source,Methodology,Standard population/values,Confidence interval details,Source of numerator,...,Indicator Content,Specific rationale,Simple Name,Simple Definition,Impact of COVID-19,Unit,Value type,Year type,Polarity,Date updated
0,93468,Proportion of GP registered populations by age...,Proportion of the GP registered population (in...,NaN,NHS England,Data are extracted each month as a snapshot in...,number of patients (in the specified age group...,NaN,NaN,"NHS England (NHSE), Patients registered at a G...",...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Financial year end point,Not applicable,22/05/2024
1,93553,Deprivation score (IMD 2019),The English Indices of Deprivation 2019 use 39...,Deprivation covers a broad range of issues and...,"English indices of deprivation 2019, Ministry ...",For LSOA deprivation scores: https://www.gov....,"For GP deprivation scores, the population weig...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Score,Calendar,RAG - Low is good,21/03/2023
2,91337,New cancer cases (Crude incidence rate),The number of persons diagnosed with any invas...,This indicator gives the total number of new c...,"NHS England, National Disease Registration Ser...",NaN,"All reportable malignant cancers, excluding no...",NaN,NaN,"NHS England (NHSE), National Disease Registrat...",...,NaN,NaN,Incidence of cancer,Rate of people diagnosed with any invasive can...,NaN,"per 100,000",Crude rate,Financial,BOB - Blue orange blue,13/12/2023
3,94063,Breast screening coverage: aged 53 to 70 years...,3-year screening coverage %: The number of eli...,Breast screening supports early detection of c...,"NHS England, Breast Screening Programme",NaN,The data were taken from the National Health A...,NaN,NaN,"NHS England (NHSE), Breast Screening Programme...",...,NaN,NaN,NaN,NaN,Breast screening coverage is low for some prac...,%,Proportion,Financial,BOB - Blue orange blue,28/11/2023
4,93725,Cervical screening coverage: aged 25 to 49 yea...,3.5-year cervical screening coverage (%): the ...,Cervical screening supports detection of cell ...,"NHS England, Cervical Screening Programme.",NaN,Data was taken from National Health Applicatio...,NaN,NaN,"NHS England (NHSE), Cervical Screening Program...",...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Financial,BOB - Blue orange blue,20/12/2023
5,93726,Cervical screening coverage: aged 50 to 64 yea...,5.5-year cervical screening coverage (%): the ...,Cervical screening supports detection of cell ...,"NHS England, Cervical Screening Programme",NaN,Data was taken from National Health Applicatio...,NaN,NaN,"NHS England (NHSE), Cervical Screening Program...",...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Financial,BOB - Blue orange blue,20/12/2023
6,92600,Bowel cancer screening coverage: aged 60 to 74...,2.5-year screening coverage (%): the number of...,Bowel cancer screening supports early detectio...,"NHS England, Bowel Cancer Screening Programme.",NaN,Data was taken from National Health Applicatio...,NaN,NaN,"NHS England (NHSE), Bowel Cancer Screening Pro...",...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Financial,BOB - Blue orange blue,08/12/2023
7,92605,Urgent suspected cancer referrals (Indirectly ...,The number of urgent suspected cancer referral...,Urgent suspected cancer referrals are a manage...,"NHS England, National Disease Registration Ser...",NaN,Referrals are assigned to a practice based on ...,NaN,NaN,"NHS England (NHSE), Cancer Waiting Times (CWT).",...,NaN,NaN,NaN,NaN,NaN,per 100,Indirectly standardised ratio,Financial,BOB - Blue orange blue,17/11/2023
8,92608,Urgent suspected cancer referrals resulting in...,The number of urgent suspected cancer referral...,Urgent suspected cancer referrals are a manage...,"NHS England, National Disease Registration Ser...",NaN,Referrals are assigned to a practice based on ...,NaN,NaN,"NHS England (NHSE), Cancer Waiting Times (CWT).",...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Financial,BOB - Blue orange blue,17/11/2023
9,92609,New can

In [ ]:
ftp.metadata.get_area_types_as_dict(is_test=False) #pulls metadata on area codes e.g data for GPs = 7

In [5]:
from urllib.parse import urlencode
import pandas as pd

def get_all_data_for_indicators(indicators, area_type_id, parent_area_type_id, filter_by_area_codes, is_test):
    base_url = 'https://fingertipsws.phe.org.uk/api/'  # Replace with your actual base URL
    endpoint = 'all_data/csv/by_indicator_id'
    
    # Construct URL with proper encoding
    params = {
        'indicator_ids': ','.join(indicators),
        'child_area_type_id': area_type_id,
        'parent_area_type_id': parent_area_type_id,
        'filter_by_area_codes': filter_by_area_codes
    }
    
    # Construct the full URL
    full_url = f"{base_url}{endpoint}?{urlencode(params)}"
    
    # Fetch data
    try:
        df = pd.read_csv(full_url)
        return df
    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle exception as needed

# Custom parameters
indicators = ['91355', '91357', '276', '91280', '91845', '92588', '93553', '91337']
area_type_id = 7
parent_area_type_id = 66
filter_by_area_codes = False
is_test = False

df = get_all_data_for_indicators(indicators, area_type_id, parent_area_type_id, filter_by_area_codes, is_test)


C:\Users\MaiaSlidel\AppData\Local\Temp\ipykernel_26992\2978097557.py:21: DtypeWarning: Columns (9,10,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(full_url)


In [6]:
ncl_icb_only = df[df['Parent Code'] == 'nE38000240'] #filtering data for NCL ICB only

In [7]:
ncl_icb_only.head()

,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to ICB sub-locations value or percentiles,Time period Sortable,New data,Compared to goal,Time period range
1898,91355,Number of emergency admissions with cancer,nE38000240,North Central London ICB - 93C,E83003,Oakleigh Road Health Centre,GPs,Persons,All ages,NaN,...,92.0,8037.0,There is a data quality issue with this value,NaN,Higher,Higher,20090000,NaN,NaN,1y
1899,91355,Number of emergency admissions with cancer,nE38000240,North Central London ICB - 93C,E83005,Lichfield Grove Surgery,GPs,Persons,All ages,NaN,...,12.0,3651.0,There is a data quality issue with this value,NaN,Similar,Similar,20090000,NaN,NaN,1y
1900,91355,Number of emergency admissions with cancer,nE38000240,North Central London ICB - 93C,E83006,Greenfield Medical Centre,GPs,Persons,All ages,NaN,...,34.0,6859.0,There is a data quality issue with this value,NaN,Similar,Similar,20090000,NaN,NaN,1y
1901,91355,Number of emergency admissions with cancer,nE38000240,North Central London ICB - 93C,E83007,Squires Lane Medical Practice,GPs,Persons,All ages,NaN,...,55.0,5582.0,There is a data quality issue with this value,NaN,Higher,Higher,20090000,NaN,NaN,1y
1902,91355,Number of emergency admissions with cancer,nE38000240,North Central London ICB - 93C,E83008,Heathfielde Medical Centre,GPs,Persons,All ages,NaN,...,33.0,7285.0,There is a data quality issue with this value,NaN,Similar,Similar,20090000,NaN,NaN,1y


In [8]:
#removing unwanted columns

usable = ncl_icb_only[['Indicator ID',
'Indicator Name',
'Area Code',
'Area Name',
'Value',
'Upper CI 99.8 limit',
'Count',
'Denominator',
'Time period Sortable',
'Time period range'
]]

usable.head()

,Indicator ID,Indicator Name,Area Code,Area Name,Value,Upper CI 99.8 limit,Count,Denominator,Time period Sortable,Time period range
1898,91355,Number of emergency admissions with cancer,E83003,Oakleigh Road Health Centre,1144.705736,1563.888648,92.0,8037.0,20090000,1y
1899,91355,Number of emergency admissions with cancer,E83005,Lichfield Grove Surgery,328.677075,741.740875,12.0,3651.0,20090000,1y
1900,91355,Number of emergency admissions with cancer,E83006,Greenfield Medical Centre,495.699081,819.240586,34.0,6859.0,20090000,1y
1901,91355,Number of emergency admissions with cancer,E83007,Squires Lane Medical Practice,985.309925,1469.433009,55.0,5582.0,20090000,1y
1902,91355,Number of emergency admissions with cancer,E83008,Heathfielde Medical Centre,452.985587,754.006745,33.0,7285.0,20090000,1y


In [9]:
#grouping by indicator name & GP code, then getting maximum date available for each - filters the data to get latest data for each indicator only
idx=usable.groupby(['Indicator Name','Area Code'])['Time period Sortable'].idxmax()
#m=max(usable['Time period Sortable'])

max_rows = usable.loc[idx]

In [10]:
#transforming dataframe into a csv file
max_rows.to_csv('Fingertips_Data.csv',index=False)

## Retrieving Social Prescribing Data

Data pre-downloaded and saved to raw data folder - SP data is emailed to us by NCL ICB and is not available in Fingertips

In [12]:
sp = pd.read_csv('Data/Raw/ICB/Cancer_Safety_Netting_Social_Prescribing_FIT/Social Prescribing/Social prescriber referrals made in the last 12 months.csv')

In [13]:
sp.head(10) #having a look at the raw data

,Social prescriber referrals made in the last 12 months,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Description,"No of referrals, offered and declined",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Parent Population,Currently registered regular patients,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Author,"NAZIMUDDIN, Nezila (Ms)",NaN,Date Modified,30-Sep-2022,NaN,NaN,NaN,NaN
4,Last Run,02-Jul-2024 11:05,NaN,Relative Date,01-Jul-2024,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Organisation,CDB,Population Count,Parent,%,Males,Females,Excluded,Status
7,THE MUSWELL HILL PRACTICE,F85063,130,16777,1%,46,84,16647,Completed
8,Highgate Group Practice,F85014,269,16883,2%,89,180,16614,Completed
9,THE BEAUMONT PRACTICE,F83671,30,3269,1%,14,16,3239,Completed


In [15]:
#promoting 6th row as header
new_header = sp.iloc[6]
sp2 = sp[1:]

In [16]:
sp2.columns = new_header
sp2.head(10)

6,Organisation,CDB,Population Count,Parent,%,Males,Females,Excluded,Status
1,Description,"No of referrals, offered and declined",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Parent Population,Currently registered regular patients,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Author,"NAZIMUDDIN, Nezila (Ms)",NaN,Date Modified,30-Sep-2022,NaN,NaN,NaN,NaN
4,Last Run,02-Jul-2024 11:05,NaN,Relative Date,01-Jul-2024,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Organisation,CDB,Population Count,Parent,%,Males,Females,Excluded,Status
7,THE MUSWELL HILL PRACTICE,F85063,130,16777,1%,46,84,16647,Completed
8,Highgate Group Practice,F85014,269,16883,2%,89,180,16614,Completed
9,THE BEAUMONT PRACTICE,F83671,30,3269,1%,14,16,3239,Completed
10,EAST BARNET HEALTH CENTRE,E83613,137,11241,1%,53,84,11104,Completed


In [35]:
#dropping the first few rows
sp3 = sp2.drop([1,2,3,4,5,6], axis='index')
sp3.head(10)

6,Organisation,CDB,Population Count,Parent,%,Males,Females,Excluded,Status
7,THE MUSWELL HILL PRACTICE,F85063,130,16777,1%,46,84,16647,Completed
8,Highgate Group Practice,F85014,269,16883,2%,89,180,16614,Completed
9,THE BEAUMONT PRACTICE,F83671,30,3269,1%,14,16,3239,Completed
10,EAST BARNET HEALTH CENTRE,E83613,137,11241,1%,53,84,11104,Completed
11,Bounds Green Group Practice,F85066,205,19027,1%,71,134,18822,Completed
12,BELSIZE PRIORY MEDICAL PRACTICE,F83658,102,5591,2%,53,49,5489,Completed
13,EAST FINCHLEY MEDICAL CENTRE,E83050,144,7524,2%,57,87,7380,Completed
14,The Crouch Hall Road Surgery,F85069,132,8391,2%,58,74,8259,Completed
15,Temple Fortune Medical Group,E83622,129,9388,1%,38,91,9259,Completed
16,Ravenscroft Medical Centre,E83039,85,5940,1%,37,48,5855,Completed


In [41]:
#dropping the total row
sp4 = sp3.drop(sp3[sp3['Organisation'] == 'Total'].index)

In [45]:
#creating new df with only required columns
sp5 = sp4[['Organisation', 'CDB', 'Population Count']]

In [61]:
#renaming the columns to match the FT data, prepping to join them vertically
sp5.rename(columns={'Organisation':'Area Name','CDB':'Area Code','Population Count':'Value'}, inplace=True)

C:\Users\MaiaSlidel\AppData\Local\Temp\ipykernel_26992\3101081217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp5.rename(columns={'Organisation':'Area Name','CDB':'Area Code','Population Count':'Value'}, inplace=True)


In [65]:
#checking if the columns in SP data are in FT, if not then populate them with NA (reason for this is that both DFs need to be the same width to perform the vertical join)
for column in max_rows.columns:
    if column not in sp5.columns:
        sp5[column] = pd.NA

sp5 = sp5[max_rows.columns]

C:\Users\MaiaSlidel\AppData\Local\Temp\ipykernel_26992\3889559910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp5[column] = pd.NA
C:\Users\MaiaSlidel\AppData\Local\Temp\ipykernel_26992\3889559910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp5[column] = pd.NA
C:\Users\MaiaSlidel\AppData\Local\Temp\ipykernel_26992\3889559910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [73]:
#setting indicator name for SP data
sp5['Indicator Name'] = 'No. of Social Prescribing referrals made within the last 12 months'

In [87]:
#extracts date from raw SP file and populates a date column
from datetime import datetime
relative_date_str = sp2.loc[sp2['Parent'] == 'Relative Date', '%'].values[0]
print(f"Relative Date: {relative_date}")

Relative Date: 01-Jul-2024


In [89]:
relative_date = datetime.strptime(relative_date_str, '%d-%b-%Y')
year = relative_date.strftime('%Y')
print(f"Year extracted: {year}")

Year extracted: 2024


In [93]:
sp5['Time period Sortable'] = year + '0000' #making SP extracted date match format in FT data

In [95]:
#commencing the merge!!!
merged = pd.concat([max_rows, sp5], ignore_index = True)
merged.head(10) 

C:\Users\MaiaSlidel\AppData\Local\Temp\ipykernel_26992\2919347274.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat([max_rows, sp5], ignore_index = True)


,Indicator ID,Indicator Name,Area Code,Area Name,Value,Upper CI 99.8 limit,Count,Denominator,Time period Sortable,Time period range
0,276,Cancer: QOF prevalence (all ages),E83003,Oakleigh Road Health Centre,4.09853,4.77773,386.0,9418.0,20220000,1y
1,276,Cancer: QOF prevalence (all ages),E83005,Lichfield Grove Surgery,2.43454,3.09697,159.0,6531.0,20220000,1y
2,276,Cancer: QOF prevalence (all ages),E83006,Greenfield Medical Centre,3.26463,3.97469,236.0,7229.0,20220000,1y
3,276,Cancer: QOF prevalence (all ages),E83007,Squires Lane Medical Practice,3.1946,4.01416,175.0,5478.0,20220000,1y
4,276,Cancer: QOF prevalence (all ages),E83008,Heathfielde Medical Centre,3.14112,3.77145,274.0,8723.0,20220000,1y
5,276,Cancer: QOF prevalence (all ages),E83009,Phgh Doctors,4.67091,5.29632,572.0,12246.0,20220000,1y
6,276,Cancer: QOF prevalence (all ages),E83010,The Speedwell Practice,4.09118,4.70008,472.0,11537.0,20220000,1y
7,276,Cancer: QOF prevalence (all ages),E83011,The Everglade Medical Practice,1.65408,2.07138,184.0,11124.0,20220000,1y
8,276,Cancer: QOF prevalence (all ages),E83012,The Old Court House Surgery,4.58897,5.31629,417.0,9087.0,20220000,1y
9,276,Cancer: QOF prevalence (all ages),E83013,Cornwall House Surgery,3.2872,4.09248,190.0,5780.0,20220000,1y


In [77]:
merged['Indicator Name'].unique() #test to see unique indicator names

array(['Cancer: QOF prevalence (all ages)',
       'Deprivation score (IMD 2019)',
       'New cancer cases (Crude incidence rate)',
       'Number of emergency admissions with cancer',
       'Number of other (non-emergency) presentations',
       'Obesity: QOF prevalence (18+ yrs)',
       'Smoking: QOF prevalence (15+ yrs)',
       'Urgent suspected cancer referrals resulting in a diagnosis of cancer (Conversion rate: as % of all USC referrals)',
       'No. of Social Prescribing referrals made within the last 12 months'],
      dtype=object)

In [129]:
#saving to prod folder
file_name = 'PCN Population Health FT_ICB.csv'
full_path  = 'Data/Production' + '/' + file_name
merged.to_csv(full_path, index=False)
print('saved file to production folder!')

saved file to production folder!


In [97]:
check = merged[merged['Indicator Name'] =='No. of Social Prescribing referrals made within the last 12 months'] 
check.head(10)
#final test to check the merge for social prescribing data

,Indicator ID,Indicator Name,Area Code,Area Name,Value,Upper CI 99.8 limit,Count,Denominator,Time period Sortable,Time period range
1416,<NA>,No. of Social Prescribing referrals made withi...,F85063,THE MUSWELL HILL PRACTICE,130,NaN,NaN,NaN,20240000,NaN
1417,<NA>,No. of Social Prescribing referrals made withi...,F85014,Highgate Group Practice,269,NaN,NaN,NaN,20240000,NaN
1418,<NA>,No. of Social Prescribing referrals made withi...,F83671,THE BEAUMONT PRACTICE,30,NaN,NaN,NaN,20240000,NaN
1419,<NA>,No. of Social Prescribing referrals made withi...,E83613,EAST BARNET HEALTH CENTRE,137,NaN,NaN,NaN,20240000,NaN
1420,<NA>,No. of Social Prescribing referrals made withi...,F85066,Bounds Green Group Practice,205,NaN,NaN,NaN,20240000,NaN
1421,<NA>,No. of Social Prescribing referrals made withi...,F83658,BELSIZE PRIORY MEDICAL PRACTICE,102,NaN,NaN,NaN,20240000,NaN
1422,<NA>,No. of Social Prescribing referrals made withi...,E83050,EAST FINCHLEY MEDICAL CENTRE,144,NaN,NaN,NaN,20240000,NaN
1423,<NA>,No. of Social Prescribing referrals made withi...,F85069,The Crouch Hall Road Surgery,132,NaN,NaN,NaN,20240000,NaN
1424,<NA>,No. of Social Prescribing referrals made withi...,E83622,Temple Fortune Medical Group,129,NaN,NaN,NaN,20240000,NaN
1425,<NA>,No. of Social Prescribing referrals made withi...,E83039,Ravenscroft Medical Centre,85,NaN,NaN,NaN,20240000,NaN
